In [5]:
#Import Modules

import os
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
import re
from docxtpl import DocxTemplate
from docx.shared import RGBColor
from datetime import datetime

In [6]:
#Briefing Note variables. Manual input required. 

variable_Prepared_for = "Dean's Council & Other Stakeholders" #Default value: "Dean's Council & Other Stakeholders"
variable_Title1 = "Keyano College Enrolment KPI Update" #Default value: "Keyano College Enrolment KPI Update"
variable_Date_prepared = "November 03, 2025" #Enter today's date. Format: "Month DD, YYYY"
variable_Prepared_by = "Aman Debesay, Joanna Duyungan" #Enter your name(s). Format: "Firstname Lastname"
variable_Title2 = "Institutional Research Analyst" #Default value: "Institutional Research Analyst"
variable_Department = "Institutional Research" #Default value: "Institutional Research"
variable_Extraction_date = "November 03, 2025" #Enter date of data extraction. Format: "Month DD, YYYY"

#Briefing Note Enrolment variables. Manual input required. 

variable_Current_enrolment_academic_year = "2025-26" #Enter the current academic year. Format: "YYYY-YY"

In [7]:
#Dictionary Mapping. Manual input required once at the start of each academic year.

#Enrolment Projection Map for the current academic year, determined before the start of the current academic year, excluding Power Engineering CML, LINC, and Apprenticeship.

enrolment_projection_map = {
    "FLE Projection All Students" : 1875.845,

    "Summer Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 0,
    "Summer Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 56,
    "Summer Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 0,
    "Summer Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 225,

    "Fall Part-time Headcount Projection All Students (Excluding CML/LINC/APPR)" : 157,
    "Fall Full-time Headcount Projection All Students (Excluding CML/LINC/APPR)" : 1816,
    "Fall Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 83,
    "Fall Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 917,
    "Fall Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 74,
    "Fall Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 899,

    "Winter Part-time Headcount Projection All Students (Excluding CML/LINC/APPR)" : 88,
    "Winter Full-time Headcount Projection All Students (Excluding CML/LINC/APPR)" : 1294,
    "Winter Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 53,
    "Winter Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)" : 813,
    "Winter Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 35,
    "Winter Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)" : 481
}

#Enrolment Projection Maps for the current academic year, for Apprenticeship only.

enrolment_projection_map_Fall_Apprenticeship = {
    "AELEC" : {
        "1" : 16,
        "2" : 16,
        "3" : 16,
        "4" : 16
    },
    "AHD" : {
        "1" : 36,
        "2" : 36,
        "3" : 18,
        "4" : 18
    },
    "AMILL" : {
        "1" : 12,
        "2" : 12,
        "3" : 0,
        "4" : 12
    },
    "ASTEAM" : {
        "1" : 12,
        "2" : 12,
        "3" : 0,
        "4" : 0
    },
    "AWELD" : {
        "1" : 12,
        "2" : 12,
        "3" : 12
    },
    "Total" : 252
}

enrolment_projection_map_Winter_Apprenticeship = {
    "AELEC" : {
        "1" : 16,
        "2" : 16,
        "3" : 16,
        "4" : 16
    },
    "AHD" : {
        "1" : 36,
        "2" : 36,
        "3" : 36,
        "4" : 18
    },
    "AMILL" : {
        "1" : 0,
        "2" : 12,
        "3" : 24,
        "4" : 0
    },
    "ASTEAM" : {
        "1" : 0,
        "2" : 0,
        "3" : 12,
        "4" : 12
    },
    "AWELD" : {
        "1" : 12,
        "2" : 12,
        "3" : 0
    },
    "Total" : 274
}

enrolment_projection_map_Spring_Apprenticeship = {
    "AELEC" : {
        "1" : 16,
        "2" : 0,
        "3" : 0,
        "4" : 16
    },
    "AHD" : {
        "1" : 18,
        "2" : 18,
        "3" : 36,
        "4" : 0
    },
    "AMILL" : {
        "1" : 12,
        "2" : 0,
        "3" : 0,
        "4" : 12
    },
    "ASTEAM" : {
        "1" : 12,
        "2" : 12,
        "3" : 0,
        "4" : 0
    },
    "AWELD" : {
        "1" : 0,
        "2" : 0,
        "3" : 12
    },
    "Total" : 164
}

#Program Map for active programs in the current academic year
active_program_map = {
    #School of University Studies, Career Programs and Academic Upgrading
    "BABUSC" : "Business Administration Certificate",
    "BAACTD" : "Business Administration Diploma - Accounting",
    "BABUSD" : "Business Administration Diploma - Management",
    "BAESPM" : "Business Administration Diploma - Esport Management",
    "BAHRMD" : "Business Administration Diploma - Human Resources Management",
    "GSCMD" : "Global Supply Chain Management Diploma",
    "BAATCO" : "Business Administration Diploma - Accounting Co-op",
    "BABSCO" : "Business Administration Diploma - Management Co-op",
    "HATM" : "Hospitality and Tourism Management Diploma",
    "HOSMNG" : "Hospitality Management Diploma",
    "NAO" : "Business Aviation Diploma (Northern Aviation Operations)",
    "ALO" : "Business Aviation Diploma (Airline Operations)",
    "BAHRMC" : "Human Resources Management Certificate",
    "OFAC" : "Office Administration Certificate",
    "HSAFA" : "Academic Foundations",
    "HSAD" : "Advanced High School Equivalency",
    "HSNP" : "College Preparation",
    "EAP" : "English For Academic Purposes",
    "HSGD" : "General High School Equivalency",
    "UTELEM" : "University Transfer: Bachelor of Elementary Education",
    "UTSEC" : "University Transfer: Bachelor of Secondary Education",
    "ENVTD" : "Environmental Technology Co-op Diploma",
    "ENVNC" : "Environmental Technology Diploma",
    "COMSCC" : "Computer Systems Technology Certificate",
    "COMSCD" : "Computer Systems Technology Diploma",
    "GNARTC" : "General Arts Certificate",
    "GNSCIC" : "General Science Certificate",
    "GOVNSC" : "Governance and Civil Studies Diploma",
    "NSPDC" : "Network Security Post-Diploma Certificate",
    "UTARTS" : "University Transfer: Bachelor of Arts",
    "UTCOMM" : "University Transfer: Bachelor of Commerce",
    "UTENG" : "University Transfer: Bachelor of Engineering",
    "UTSCGS" : "University Transfer: Bachelor of Science",
    "UTSCWK" : "University Transfer: Bachelor of Social Work",
    "OPEN" : "Open Studies",
    "AITS" : "Applied IT Specialist",

    #School of Health & Human Services 
    "AELCC" : "Applied Early Learning and Child Care Certificate",
    "ECCERT" : "Early Learning and Child Care Certificate",
    "ECDIP" : "Early Learning and Child Care Diploma",
    "CYCD" : "Child and Youth Care Diploma",
    "ECEAC" : "Educational Assistant Certificate",
    "SOCWKD" : "Social Work Diploma",
    "HCAC" : "Health Care Aide",
    "ACP" : "Advanced Care Paramedic",
    "PCPC" : "Primary Care Paramedic",
    "PNR" : "Practical Nurse Diploma",
    "BSCN" : "University Transfer: Bachelor of Nursing",
    "BSCACN" : "University Transfer: Bachelor of Nursing",
    "BSCAFT" : "University Transfer: Bachelor of Nursing",
    "BSCAEE" : "University Transfer: Bachelor of Nursing",
    "IENCN" : "IEN Bridge to Canadian Nursing",

    #School of Trades and Heavy Industrial
    "AELEC" : "Apprenticeship Electrician",
    "AHD" : "Apprenticeship Heavy Equipment Technician",
    "AMILL" : "Apprenticeship Industrial Mechanic (Millwright)",
    "ASTEAM" : "Apprenticeship Steamfitter-Pipefitter",
    "AWELD" : "Apprenticeship Welder",
    "POWCM3" : "Power Engineering 3rd Class CML",
    "POWCM4" : "Power Engineering 4th Class CML",
    "POWCO3" : "Power Engineering 3rd Class Co-op",
    "POWCO4" : "Power Engineering 4th Class Co-op",
    "POWCP4" : "Power Engineering 4th Class Comprehensive",
    "PPET" : "Power and Process Technologies",
    "PACARP" : "Pre-employment Carpentry",
    "PAELEC" : "Pre-employment Electrician",
    "PAHET" : "Pre-employment HET",
    "PAPLUM" : "Pre-employment Gasfitter",
    "PAWELD" : "Pre-employment Welder",

    #Language Instruction for Newcomers to Canada (LINC)
    "LINC13" : "Language Instruction for Newcomers to Canada"
}

#Year (CLASS_LEVEL) Map

class_level_map = {
    "1" : "First year",
    "2" : "Second year",
    "3" : "Third year",
    "4" : "Fourth year"
}


In [8]:
#Search in the folder 'Data Files' for 4 application comparative files, and assign them to 4 variables chronologically

term_order = {"Winter": 1, "Spring": 2, "Summer": 3, "Fall": 4}

def extract_term_and_year(filename):
    match = re.search(r"(Winter|Spring|Summer|Fall)\s*(\d{4})", filename, re.IGNORECASE)
    if match:
        term = match.group(1).capitalize()
        year = int(match.group(2))
        return (year, term_order[term])
    return None

def get_chronological_application_comparative_files(folder_path):
    files = os.listdir(folder_path)
    matched_files = []

    for file in files:
        if not file.lower().endswith(".xlsx"):
            continue
        if re.search(r"applications?\s*comparative", file, re.IGNORECASE):
            term_info = extract_term_and_year(file)
            if term_info:
                matched_files.append((term_info, os.path.join(folder_path, file)))

    if len(matched_files) != 4:
        raise ValueError(f"Expected exactly 4 'Application Comparative' files, found {len(matched_files)}.")

    matched_files.sort(key=lambda x: (x[0][0], x[0][1]))

    return [f[1] for f in matched_files]

data_folder = "Data Files"

(
    term_1_file_application_comparative_filename,
    term_2_file_application_comparative_filename,
    term_3_file_application_comparative_filename,
    term_4_file_application_comparative_filename,
) = get_chronological_application_comparative_files(data_folder)

print("Term 1 Application Comparative:", term_1_file_application_comparative_filename)
print("Term 2 Application Comparative:", term_2_file_application_comparative_filename)
print("Term 3 Application Comparative:", term_3_file_application_comparative_filename)
print("Term 4 Application Comparative:", term_4_file_application_comparative_filename)

Term 1 Application Comparative: Data Files\zTestingData-Application Comparative -Summer 2025 as of June 02, 2025.xlsx
Term 2 Application Comparative: Data Files\zTestingData-Application Comparative -Fall 2025 as of June 02, 2025.xlsx
Term 3 Application Comparative: Data Files\zTestingData-Application Comparative -Winter 2026 as of June 02, 2025.xlsx
Term 4 Application Comparative: Data Files\zTestingData-Application Comparative -Spring 2026 as of June 02, 2025.xlsx


In [9]:
#Search in the folder 'Data Files' for 4 application details files, and assign them to 4 variables chronologically

term_order = {"Winter": 1, "Spring": 2, "Summer": 3, "Fall": 4}

def extract_term_and_year(filename):
    match = re.search(r"(Winter|Spring|Summer|Fall)\s*(\d{4})", filename, re.IGNORECASE)
    if match:
        term = match.group(1).capitalize()
        year = int(match.group(2))
        return (year, term_order[term])
    return None

def get_chronological_applications_reports_details_files(folder_path):
    files = os.listdir(folder_path)
    matched_files = []

    for file in files:
        if not file.lower().endswith(".xlsx"):
            continue
        if re.search(r"applications?\s*reports?\s*details?", file, re.IGNORECASE):
            term_info = extract_term_and_year(file)
            if term_info:
                matched_files.append((term_info, os.path.join(folder_path, file)))

    if len(matched_files) != 4:
        raise ValueError(f"Expected exactly 4 'Application Comparative' files, found {len(matched_files)}.")

    matched_files.sort(key=lambda x: (x[0][0], x[0][1]))

    return [f[1] for f in matched_files]

data_folder = "Data Files"

(
    term_1_file_application_details_filename,
    term_2_file_application_details_filename,
    term_3_file_application_details_filename,
    term_4_file_application_details_filename,
) = get_chronological_applications_reports_details_files(data_folder)

print("Term 1 Applications Reports Details:", term_1_file_application_details_filename)
print("Term 2 Applications Reports Details:", term_2_file_application_details_filename)
print("Term 3 Applications Reports Details:", term_3_file_application_details_filename)
print("Term 4 Applications Reports Details:", term_4_file_application_details_filename)

Term 1 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Summer 2025 as of June 02, 2025.xlsx
Term 2 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Fall 2025 as of June 02, 2025.xlsx
Term 3 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Winter 2026 as of June 02, 2025.xlsx
Term 4 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Spring 2026 as of June 02, 2025.xlsx


In [10]:
# Search in the folder 'Data Files' for the FLE file, and assign it to a variable

def get_fle_file(folder_path):
    files = os.listdir(folder_path)
    matched_files = []

    for file in files:
        if file.lower().endswith(".xlsx") and re.search(r"fle", file, re.IGNORECASE):
            matched_files.append(os.path.join(folder_path, file))

    if len(matched_files) != 1:
        raise ValueError(f"Expected exactly 1 'FLE' file, found {len(matched_files)}.")

    return matched_files[0]

data_folder = "Data Files"

enrolment_file_current_year_filename = get_fle_file(data_folder)

print("FLE:", enrolment_file_current_year_filename)

FLE: Data Files\FLE as of October 15, 2025.xlsx


In [11]:
#Set filenames as variabless

term_1_file_application_comparative = term_1_file_application_comparative_filename
term_1_file_application_details = term_1_file_application_details_filename

term_2_file_application_comparative = term_2_file_application_comparative_filename
term_2_file_application_details = term_2_file_application_details_filename

term_3_file_application_comparative = term_3_file_application_comparative_filename
term_3_file_application_details = term_3_file_application_details_filename

term_4_file_application_comparative = term_4_file_application_comparative_filename
term_4_file_application_details = term_4_file_application_details_filename 

enrolment_file_current_year = enrolment_file_current_year_filename
enrolment_file_previous_year_summary = "Briefing Note Enrolment Comparison Table From Previous Academic Year.xlsx" #No need to change by default, unless file name is changed


In [12]:
#Customized Math Formula Functions

#Function for subtracting current_year_value by previous_year_value, then dividing by previous_year_value, then multiplying by 100, rounding to 1 decimal place
def percent_difference_value_calculator_round_1(current_year_value, previous_year_value):
    try:
        return round((current_year_value - previous_year_value) / previous_year_value * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"
    
#Function for dividing numerator by denominator, then multiplying by 100, rounding to 1 decimal place
def division_calculator(numerator, denominator): 
    try:
        return round(numerator / denominator * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"


In [13]:
#Application Comparative Term 1
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_1_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_1_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_1_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_1_year_current = int(re.search(r"\d{4}", term_1_term_year_current).group())
term_1_year_minus_one = term_1_year_current - 1
term_1_year_minus_two = term_1_year_current - 2
term_1_year_minus_three = term_1_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_1_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_1_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_1_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_1_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_1_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_1_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_1_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_1_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_1_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_1_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_1_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_1_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_1_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_1_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_1_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_1_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_one:
            term_1_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_merged_range}")

# Now term_1_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Unique_merged_range}")

# Now term_1_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Total_Apps_merged_range}")

# Now term_1_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_1_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_1_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_two:
            term_1_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_merged_range}")

# Now term_1_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Unique_merged_range}")

# Now term_1_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Total_Apps_merged_range}")

# Now term_1_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_three:
            term_1_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_merged_range}")

# Now term_1_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Unique_merged_range}")

# Now term_1_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Total_Apps_merged_range}")

# Now term_1_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_1_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_1_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_one_Unique_All_Applicants," ",term_1_year_minus_one_Unique_International_Applicants," ",term_1_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_one_Total_Apps_All_Applicants," ",term_1_year_minus_one_Total_Apps_International_Applicants," ",term_1_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_two_Unique_All_Applicants," ",term_1_year_minus_two_Unique_International_Applicants," ",term_1_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_two_Total_Apps_All_Applicants," ",term_1_year_minus_two_Total_Apps_International_Applicants," ",term_1_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK-------------------------------------------------------------------------------term_1--------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_1_year_minus_three_Unique_All_Applicants," ",term_1_year_minus_three_Unique_International_Applicants," ",term_1_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_1_year_minus_three_Total_Apps_All_Applicants," ",term_1_year_minus_three_Total_Apps_International_Applicants," ",term_1_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_1_term_year_current)

if term_match:
    term_1_term = term_match.group(1)
    term_1_year_current = int(term_match.group(2))

    # Subtract one year
    term_1_year_minus_one = term_1_year_current - 1

    # Construct the previous term-year string
    term_1_term_year_minus_one = f"{term_1_term} {term_1_year_minus_one}"

#Variables created
print("Current Term and Year:", term_1_term_year_current)
print("Current Term and Year Minus 1:", term_1_term_year_minus_one)
print("\n")
print(f"{term_1_year_minus_one}, Unique, All Applicants:", term_1_year_minus_one_Unique_All_Applicants)
print(f"{term_1_year_minus_one}, Unique, International Applicants:", term_1_year_minus_one_Unique_International_Applicants)
print(f"{term_1_year_minus_one}, Unique, Domestic Applicants:", term_1_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, All Applicants:", term_1_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, International Applicants:", term_1_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, Domestic Applicants:", term_1_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_two}, Unique, All Applicants:", term_1_year_minus_two_Unique_All_Applicants)
print(f"{term_1_year_minus_two}, Unique, International Applicants:", term_1_year_minus_two_Unique_International_Applicants)
print(f"{term_1_year_minus_two}, Unique, Domestic Applicants:", term_1_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, All Applicants:", term_1_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, International Applicants:", term_1_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, Domestic Applicants:", term_1_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_three}, Unique, All Applicants:", term_1_year_minus_three_Unique_All_Applicants)
print(f"{term_1_year_minus_three}, Unique, International Applicants:", term_1_year_minus_three_Unique_International_Applicants)
print(f"{term_1_year_minus_three}, Unique, Domestic Applicants:", term_1_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, All Applicants:", term_1_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, International Applicants:", term_1_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, Domestic Applicants:", term_1_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Summer 2025
Current Term and Year Minus 1: Summer 2024


2024, Unique, All Applicants: 1425
2024, Unique, International Applicants: 1387
2024, Unique, Domestic Applicants: 38
2024, Total Apps, All Applicants: 1431
2024, Total Apps, International Applicants: 1393
2024, Total Apps, Domestic Applicants: 38


2023, Unique, All Applicants: 39
2023, Unique, International Applicants: 1
2023, Unique, Domestic Applicants: 38
2023, Total Apps, All Applicants: 39
2023, Total Apps, International Applicants: 1
2023, Total Apps, Domestic Applicants: 38


2022, Unique, All Applicants: 42
2022, Unique, International Applicants: 0
2022, Unique, Domestic Applicants: 42
2022, Total Apps, All Applicants: 42
2022, Total Apps, International Applicants: 0
2022, Total Apps, Domestic Applicants: 42


In [14]:
#Application Comparative Term 2
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_2_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_2_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_2_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_2_year_current = int(re.search(r"\d{4}", term_2_term_year_current).group())
term_2_year_minus_one = term_2_year_current - 1
term_2_year_minus_two = term_2_year_current - 2
term_2_year_minus_three = term_2_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_2_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_2_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_2_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_2_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_2_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_2_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_2_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_2_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_2_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_2_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_2_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_2_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_2_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_2_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_2_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_2_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_one:
            term_2_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_merged_range}")

# Now term_2_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Unique_merged_range}")

# Now term_2_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Total_Apps_merged_range}")

# Now term_2_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_2_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_2_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_two:
            term_2_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_merged_range}")

# Now term_2_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Unique_merged_range}")

# Now term_2_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Total_Apps_merged_range}")

# Now term_2_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_three:
            term_2_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_merged_range}")

# Now term_2_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Unique_merged_range}")

# Now term_2_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Total_Apps_merged_range}")

# Now term_2_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_2_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_2_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_one_Unique_All_Applicants," ",term_2_year_minus_one_Unique_International_Applicants," ",term_2_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_one_Total_Apps_All_Applicants," ",term_2_year_minus_one_Total_Apps_International_Applicants," ",term_2_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_two_Unique_All_Applicants," ",term_2_year_minus_two_Unique_International_Applicants," ",term_2_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_two_Total_Apps_All_Applicants," ",term_2_year_minus_two_Total_Apps_International_Applicants," ",term_2_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_2_year_minus_three_Unique_All_Applicants," ",term_2_year_minus_three_Unique_International_Applicants," ",term_2_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_2_year_minus_three_Total_Apps_All_Applicants," ",term_2_year_minus_three_Total_Apps_International_Applicants," ",term_2_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_2_term_year_current)

if term_match:
    term_2_term = term_match.group(1)
    term_2_year_current = int(term_match.group(2))

    # Subtract one year
    term_2_year_minus_one = term_2_year_current - 1

    # Construct the previous term-year string
    term_2_term_year_minus_one = f"{term_2_term} {term_2_year_minus_one}"

#Variables created
print("Current Term and Year:", term_2_term_year_current)
print("Current Term and Year Minus 1:", term_2_term_year_minus_one)
print("\n")
print(f"{term_2_year_minus_one}, Unique, All Applicants:", term_2_year_minus_one_Unique_All_Applicants)
print(f"{term_2_year_minus_one}, Unique, International Applicants:", term_2_year_minus_one_Unique_International_Applicants)
print(f"{term_2_year_minus_one}, Unique, Domestic Applicants:", term_2_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, All Applicants:", term_2_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, International Applicants:", term_2_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, Domestic Applicants:", term_2_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_two}, Unique, All Applicants:", term_2_year_minus_two_Unique_All_Applicants)
print(f"{term_2_year_minus_two}, Unique, International Applicants:", term_2_year_minus_two_Unique_International_Applicants)
print(f"{term_2_year_minus_two}, Unique, Domestic Applicants:", term_2_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, All Applicants:", term_2_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, International Applicants:", term_2_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, Domestic Applicants:", term_2_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_three}, Unique, All Applicants:", term_2_year_minus_three_Unique_All_Applicants)
print(f"{term_2_year_minus_three}, Unique, International Applicants:", term_2_year_minus_three_Unique_International_Applicants)
print(f"{term_2_year_minus_three}, Unique, Domestic Applicants:", term_2_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, All Applicants:", term_2_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, International Applicants:", term_2_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, Domestic Applicants:", term_2_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Fall 2025
Current Term and Year Minus 1: Fall 2024


2024, Unique, All Applicants: 4369
2024, Unique, International Applicants: 3508
2024, Unique, Domestic Applicants: 861
2024, Total Apps, All Applicants: 4655
2024, Total Apps, International Applicants: 3684
2024, Total Apps, Domestic Applicants: 971


2023, Unique, All Applicants: 2479
2023, Unique, International Applicants: 1632
2023, Unique, Domestic Applicants: 847
2023, Total Apps, All Applicants: 2713
2023, Total Apps, International Applicants: 1755
2023, Total Apps, Domestic Applicants: 958


2022, Unique, All Applicants: 1645
2022, Unique, International Applicants: 829
2022, Unique, Domestic Applicants: 816
2022, Total Apps, All Applicants: 1778
2022, Total Apps, International Applicants: 881
2022, Total Apps, Domestic Applicants: 897


In [15]:
#Application Comparative Term 3
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_3_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_3_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_3_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_3_year_current = int(re.search(r"\d{4}", term_3_term_year_current).group())
term_3_year_minus_one = term_3_year_current - 1
term_3_year_minus_two = term_3_year_current - 2
term_3_year_minus_three = term_3_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_3_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_3_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_3_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_3_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_3_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_3_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_3_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_3_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_3_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_3_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_3_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_3_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_3_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_3_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_3_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_3_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_one:
            term_3_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_merged_range}")

# Now term_3_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Unique_merged_range}")

# Now term_3_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Total_Apps_merged_range}")

# Now term_3_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_3_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_3_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_two:
            term_3_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_merged_range}")

# Now term_3_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Unique_merged_range}")

# Now term_3_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Total_Apps_merged_range}")

# Now term_3_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_three:
            term_3_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_merged_range}")

# Now term_3_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Unique_merged_range}")

# Now term_3_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Total_Apps_merged_range}")

# Now term_3_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_3_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_3_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_one_Unique_All_Applicants," ",term_3_year_minus_one_Unique_International_Applicants," ",term_3_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_one_Total_Apps_All_Applicants," ",term_3_year_minus_one_Total_Apps_International_Applicants," ",term_3_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_two_Unique_All_Applicants," ",term_3_year_minus_two_Unique_International_Applicants," ",term_3_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_two_Total_Apps_All_Applicants," ",term_3_year_minus_two_Total_Apps_International_Applicants," ",term_3_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_3_year_minus_three_Unique_All_Applicants," ",term_3_year_minus_three_Unique_International_Applicants," ",term_3_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_3_year_minus_three_Total_Apps_All_Applicants," ",term_3_year_minus_three_Total_Apps_International_Applicants," ",term_3_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_3_term_year_current)

if term_match:
    term_3_term = term_match.group(1)
    term_3_year_current = int(term_match.group(2))

    # Subtract one year
    term_3_year_minus_one = term_3_year_current - 1

    # Construct the previous term-year string
    term_3_term_year_minus_one = f"{term_3_term} {term_3_year_minus_one}"

#Variables created
print("Current Term and Year:", term_3_term_year_current)
print("Current Term and Year Minus 1:", term_3_term_year_minus_one)
print("\n")
print(f"{term_3_year_minus_one}, Unique, All Applicants:", term_3_year_minus_one_Unique_All_Applicants)
print(f"{term_3_year_minus_one}, Unique, International Applicants:", term_3_year_minus_one_Unique_International_Applicants)
print(f"{term_3_year_minus_one}, Unique, Domestic Applicants:", term_3_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, All Applicants:", term_3_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, International Applicants:", term_3_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, Domestic Applicants:", term_3_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_two}, Unique, All Applicants:", term_3_year_minus_two_Unique_All_Applicants)
print(f"{term_3_year_minus_two}, Unique, International Applicants:", term_3_year_minus_two_Unique_International_Applicants)
print(f"{term_3_year_minus_two}, Unique, Domestic Applicants:", term_3_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, All Applicants:", term_3_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, International Applicants:", term_3_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, Domestic Applicants:", term_3_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_three}, Unique, All Applicants:", term_3_year_minus_three_Unique_All_Applicants)
print(f"{term_3_year_minus_three}, Unique, International Applicants:", term_3_year_minus_three_Unique_International_Applicants)
print(f"{term_3_year_minus_three}, Unique, Domestic Applicants:", term_3_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, All Applicants:", term_3_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, International Applicants:", term_3_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, Domestic Applicants:", term_3_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Winter 2026
Current Term and Year Minus 1: Winter 2025


2025, Unique, All Applicants: 994
2025, Unique, International Applicants: 972
2025, Unique, Domestic Applicants: 22
2025, Total Apps, All Applicants: 1007
2025, Total Apps, International Applicants: 985
2025, Total Apps, Domestic Applicants: 22


2024, Unique, All Applicants: 326
2024, Unique, International Applicants: 289
2024, Unique, Domestic Applicants: 37
2024, Total Apps, All Applicants: 331
2024, Total Apps, International Applicants: 293
2024, Total Apps, Domestic Applicants: 38


2023, Unique, All Applicants: 115
2023, Unique, International Applicants: 70
2023, Unique, Domestic Applicants: 45
2023, Total Apps, All Applicants: 116
2023, Total Apps, International Applicants: 71
2023, Total Apps, Domestic Applicants: 45


In [16]:
#Application Comparative Term 4
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_4_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_4_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_4_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_4_year_current = int(re.search(r"\d{4}", term_4_term_year_current).group())
term_4_year_minus_one = term_4_year_current - 1
term_4_year_minus_two = term_4_year_current - 2
term_4_year_minus_three = term_4_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_4_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_4_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_4_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_4_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_4_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_4_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_4_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_4_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_4_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_4_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_4_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_4_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_4_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_4_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_4_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_4_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_one:
            term_4_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_merged_range}")

# Now term_4_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Unique_merged_range}")

# Now term_4_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Total_Apps_merged_range}")

# Now term_4_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_4_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_4_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_two:
            term_4_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_merged_range}")

# Now term_4_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Unique_merged_range}")

# Now term_4_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Total_Apps_merged_range}")

# Now term_4_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_three:
            term_4_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_merged_range}")

# Now term_4_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Unique_merged_range}")

# Now term_4_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Total_Apps_merged_range}")

# Now term_4_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_4_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_4_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_one_Unique_All_Applicants," ",term_4_year_minus_one_Unique_International_Applicants," ",term_4_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_one_Total_Apps_All_Applicants," ",term_4_year_minus_one_Total_Apps_International_Applicants," ",term_4_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_two_Unique_All_Applicants," ",term_4_year_minus_two_Unique_International_Applicants," ",term_4_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_two_Total_Apps_All_Applicants," ",term_4_year_minus_two_Total_Apps_International_Applicants," ",term_4_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_4_year_minus_three_Unique_All_Applicants," ",term_4_year_minus_three_Unique_International_Applicants," ",term_4_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_4_year_minus_three_Total_Apps_All_Applicants," ",term_4_year_minus_three_Total_Apps_International_Applicants," ",term_4_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_4_term_year_current)

if term_match:
    term_4_term = term_match.group(1)
    term_4_year_current = int(term_match.group(2))

    # Subtract one year
    term_4_year_minus_one = term_4_year_current - 1

    # Construct the previous term-year string
    term_4_term_year_minus_one = f"{term_4_term} {term_4_year_minus_one}"

#Variables created
print("Current Term and Year:", term_4_term_year_current)
print("Current Term and Year Minus 1:", term_4_term_year_minus_one)
print("\n")
print(f"{term_4_year_minus_one}, Unique, All Applicants:", term_4_year_minus_one_Unique_All_Applicants)
print(f"{term_4_year_minus_one}, Unique, International Applicants:", term_4_year_minus_one_Unique_International_Applicants)
print(f"{term_4_year_minus_one}, Unique, Domestic Applicants:", term_4_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, All Applicants:", term_4_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, International Applicants:", term_4_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, Domestic Applicants:", term_4_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_two}, Unique, All Applicants:", term_4_year_minus_two_Unique_All_Applicants)
print(f"{term_4_year_minus_two}, Unique, International Applicants:", term_4_year_minus_two_Unique_International_Applicants)
print(f"{term_4_year_minus_two}, Unique, Domestic Applicants:", term_4_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, All Applicants:", term_4_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, International Applicants:", term_4_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, Domestic Applicants:", term_4_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_three}, Unique, All Applicants:", term_4_year_minus_three_Unique_All_Applicants)
print(f"{term_4_year_minus_three}, Unique, International Applicants:", term_4_year_minus_three_Unique_International_Applicants)
print(f"{term_4_year_minus_three}, Unique, Domestic Applicants:", term_4_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, All Applicants:", term_4_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, International Applicants:", term_4_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, Domestic Applicants:", term_4_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Spring 2026
Current Term and Year Minus 1: Spring 2025


2025, Unique, All Applicants: 2
2025, Unique, International Applicants: 0
2025, Unique, Domestic Applicants: 2
2025, Total Apps, All Applicants: 2
2025, Total Apps, International Applicants: 0
2025, Total Apps, Domestic Applicants: 2


2024, Unique, All Applicants: 3
2024, Unique, International Applicants: 3
2024, Unique, Domestic Applicants: 0
2024, Total Apps, All Applicants: 3
2024, Total Apps, International Applicants: 3
2024, Total Apps, Domestic Applicants: 0


2023, Unique, All Applicants: 2
2023, Unique, International Applicants: 1
2023, Unique, Domestic Applicants: 1
2023, Total Apps, All Applicants: 2
2023, Total Apps, International Applicants: 1
2023, Total Apps, Domestic Applicants: 1


In [17]:
#Application Details Term 1
# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_1_details_total_applications = df_term_1_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_1_details_total_unique_applicants = df_term_1_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_1_details_total_applications}")
print(f"Total unique applicants: {term_1_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_1_details_domestic_applications = df_term_1_details[df_term_1_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_1_details_international_applications = df_term_1_details[df_term_1_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_1_details_domestic_applications}")
print(f"Total International applications: {term_1_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_1_details = df_term_1_details[
    df_term_1_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_1_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_1_details_accepted_applications = len(filtered_df_term_1_details)
print(f"Valid application count: {term_1_details_accepted_applications}")

Total applications: 50
Total unique applicants: 50
Total Domestic applications: 24
Total International applications: 26
Valid application count: 48


In [18]:
#Application Details Term 2
# Load the workbook
wb = openpyxl.load_workbook(term_2_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_2_details = pd.read_excel(term_2_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_2_details_total_applications = df_term_2_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_2_details_total_unique_applicants = df_term_2_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_2_details_total_applications}")
print(f"Total unique applicants: {term_2_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_2_details_domestic_applications = df_term_2_details[df_term_2_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_2_details_international_applications = df_term_2_details[df_term_2_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_2_details_domestic_applications}")
print(f"Total International applications: {term_2_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_2_details = df_term_2_details[
    df_term_2_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_2_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_2_details_accepted_applications = len(filtered_df_term_2_details)
print(f"Valid application count: {term_2_details_accepted_applications}")

Total applications: 1297
Total unique applicants: 1297
Total Domestic applications: 856
Total International applications: 441
Valid application count: 489


In [19]:
#Application Details Term 3
# Load the workbook
wb = openpyxl.load_workbook(term_3_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_3_details = pd.read_excel(term_3_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_3_details_total_applications = df_term_3_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_3_details_total_unique_applicants = df_term_3_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_3_details_total_applications}")
print(f"Total unique applicants: {term_3_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_3_details_domestic_applications = df_term_3_details[df_term_3_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_3_details_international_applications = df_term_3_details[df_term_3_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_3_details_domestic_applications}")
print(f"Total International applications: {term_3_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_3_details = df_term_3_details[
    df_term_3_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_3_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_3_details_accepted_applications = len(filtered_df_term_3_details)
print(f"Valid application count: {term_3_details_accepted_applications}")

Total applications: 33
Total unique applicants: 33
Total Domestic applications: 12
Total International applications: 21
Valid application count: 10


In [20]:
#Application Details Term 4
# Load the workbook
wb = openpyxl.load_workbook(term_4_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_4_details = pd.read_excel(term_4_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_4_details_total_applications = df_term_4_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_4_details_total_unique_applicants = df_term_4_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_4_details_total_applications}")
print(f"Total unique applicants: {term_4_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_4_details_domestic_applications = df_term_4_details[df_term_4_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_4_details_international_applications = df_term_4_details[df_term_4_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_4_details_domestic_applications}")
print(f"Total International applications: {term_4_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_4_details = df_term_4_details[
    df_term_4_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_4_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_4_details_accepted_applications = len(filtered_df_term_4_details)
print(f"Valid application count: {term_4_details_accepted_applications}")

Total applications: 1
Total unique applicants: 1
Total Domestic applications: 1
Total International applications: 0
Valid application count: 0


In [21]:
#Applications Calculations

#Calculate % Change for term 1 Total Applications
term_1_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)}%"
term_1_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)
try:
    term_1_total_applications_percent_change_number_absolute = np.abs(term_1_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Unique Applications
term_1_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)}%"
term_1_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)
try:
    term_1_total_applications_percent_change_number_unique_absolute = np.abs(term_1_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 1 International Applications
term_1_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)}%"
term_1_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)
try:
    term_1_international_applications_percent_change_number_absolute = np.abs(term_1_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Domestic Applications
term_1_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_1_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_1_domestic_applications_percent_change_number_absolute = np.abs(term_1_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_1}}
try:
    if term_1_total_applications_percent_change_number_unique >= 0:
        higher_lower_1 = "higher"
    else:
        higher_lower_1 = "lower"
except (TypeError, ValueError): 
    higher_lower_1 = "FAILED TO CALCULATE"

#Calculation for {{term_1_acceptance_percentage_number}}

term_1_acceptance_percentage_number = division_calculator(term_1_details_accepted_applications, term_1_details_total_applications)

#Calculation for {{term_1_international_percentage_number}}

term_1_international_percentage_number = division_calculator(term_1_details_international_applications, term_1_details_total_applications)

#Calculate % Change for term 2 Total Applications
term_2_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)}%"
term_2_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)
try:
    term_2_total_applications_percent_change_number_absolute = np.abs(term_2_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Unique Applications
term_2_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)}%"
term_2_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)
try:
    term_2_total_applications_percent_change_number_unique_absolute = np.abs(term_2_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 2 International Applications
term_2_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)}%"
term_2_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)
try:
    term_2_international_applications_percent_change_number_absolute = np.abs(term_2_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Domestic Applications
term_2_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_2_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_2_domestic_applications_percent_change_number_absolute = np.abs(term_2_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_2}}
try:
    if term_2_total_applications_percent_change_number_unique >= 0:
        higher_lower_2 = "higher"
    else:
        higher_lower_2 = "lower"
except (TypeError, ValueError): 
    higher_lower_2 = "FAILED TO CALCULATE"

#Calculation for {{term_2_acceptance_percentage_number}}

term_2_acceptance_percentage_number = division_calculator(term_2_details_accepted_applications, term_2_details_total_applications)

#Calculation for {{term_2_international_percentage_number}}

term_2_international_percentage_number = division_calculator(term_2_details_international_applications, term_2_details_total_applications)

#Calculate % Change for term 3 Total Applications
term_3_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)}%"
term_3_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)
try:
    term_3_total_applications_percent_change_number_absolute = np.abs(term_3_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Unique Applications
term_3_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)}%"
term_3_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)
try:
    term_3_total_applications_percent_change_number_unique_absolute = np.abs(term_3_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 3 International Applications
term_3_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)}%"
term_3_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)
try:
    term_3_international_applications_percent_change_number_absolute = np.abs(term_3_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Domestic Applications
term_3_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_3_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_3_domestic_applications_percent_change_number_absolute = np.abs(term_3_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_3}}
try:
    if term_3_total_applications_percent_change_number_unique >= 0:
        higher_lower_3 = "higher"
    else:
        higher_lower_3 = "lower"

except (TypeError, ValueError): 
    higher_lower_3 = "FAILED TO CALCULATE"

#Calculation for {{term_3_acceptance_percentage_number}}

term_3_acceptance_percentage_number = division_calculator(term_3_details_accepted_applications, term_3_details_total_applications)

#Calculation for {{term_3_international_percentage_number}}

term_3_international_percentage_number = division_calculator(term_3_details_international_applications, term_3_details_total_applications)

#Calculate % Change for term 4 Total Applications
term_4_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)}%"
term_4_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)
try:
    term_4_total_applications_percent_change_number_absolute = np.abs(term_4_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Unique Applications
term_4_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)}%"
term_4_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)
try:
    term_4_total_applications_percent_change_number_unique_absolute = np.abs(term_4_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 4 International Applications
term_4_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)}%"
term_4_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)
try:
    term_4_international_applications_percent_change_number_absolute = np.abs(term_4_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Domestic Applications
term_4_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_4_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_4_domestic_applications_percent_change_number_absolute = np.abs(term_4_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_domestic_applications_percent_change_number_absolute = "N/A"



In [22]:
#Current Academic Year Enrolment Data and Calculations

#Convert variable_Extraction_date variable from the "Month DD, YYYY" format into the "YYYY-MM-DD" format
parsed_Extraction_date = datetime.strptime(variable_Extraction_date, "%B %d, %Y")
converted_Extraction_date = parsed_Extraction_date.strftime("%Y-%m-%d")

#Extract year numbers from variable_Current_enrolment_academic_year for mathematical manipulations
enrolment_start_year = int(variable_Current_enrolment_academic_year.split("-")[0])
enrolment_end_year = int(variable_Current_enrolment_academic_year.split("-")[0]) + 1

# Load the Enrolment workbook
wb = openpyxl.load_workbook(enrolment_file_current_year, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_enrolment = pd.read_excel(enrolment_file_current_year, sheet_name=matched_sheet_name)

# Force CLASS_LEVEL to int, safely handling missing values
df_enrolment['CLASS_LEVEL'] = pd.to_numeric(df_enrolment['CLASS_LEVEL'], errors='coerce').astype('Int64')

#Determine the Total UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Total = df_enrolment[
    df_enrolment['FULL_PART'].str.upper().isin(['FULL', 'PART'])
]
df_enrolment_filtered_full_part_Total['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Total['FLE'], errors='coerce')

enrolment_Total_UHC = df_enrolment_filtered_full_part_Total['PEOPLE_CODE_ID'].nunique()
enrolment_Total_FLE = df_enrolment_filtered_full_part_Total['FLE'].sum()

#Determine the International UHC and FLE for the current academic year
df_enrolment_filtered_full_part_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA']))
]
df_enrolment_filtered_full_part_International['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_International['FLE'], errors='coerce')

enrolment_International_UHC = df_enrolment_filtered_full_part_International['PEOPLE_CODE_ID'].nunique()
enrolment_International_FLE = df_enrolment_filtered_full_part_International['FLE'].sum()

#Determine the Domestic UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE']))
]
df_enrolment_filtered_full_part_Domestic['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Domestic['FLE'], errors='coerce')

enrolment_Domestic_UHC = df_enrolment_filtered_full_part_Domestic['PEOPLE_CODE_ID'].nunique()
enrolment_Domestic_FLE = df_enrolment_filtered_full_part_Domestic['FLE'].sum()

#Determine the Indigenous UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Indigenous = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['HERITAGE'].str.replace(' ','').str.upper().isin(['INUIT', 'METIS', 'NONSTATUS', 'STATUSINDIAN']))
]
df_enrolment_filtered_full_part_Indigenous['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Indigenous['FLE'], errors='coerce')

enrolment_Indigenous_UHC = df_enrolment_filtered_full_part_Indigenous['PEOPLE_CODE_ID'].nunique()
enrolment_Indigenous_FLE = df_enrolment_filtered_full_part_Indigenous['FLE'].sum()

#Determine the Apprenticeship UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Apprenticeship = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['DEPARTMENT'].str.replace(' ','').str.upper().isin(['APPRENTICE']))
]
df_enrolment_filtered_full_part_Apprenticeship['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Apprenticeship['FLE'], errors='coerce')

enrolment_Apprenticeship_UHC = df_enrolment_filtered_full_part_Apprenticeship['PEOPLE_CODE_ID'].nunique()
enrolment_Apprenticeship_FLE = df_enrolment_filtered_full_part_Apprenticeship['FLE'].sum()

#Determine the Certificate UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Certificate = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['CREDENTIAL_TYPE'].str.replace(' ','').str.upper().isin(['CERTIFICATE']))
]
df_enrolment_filtered_full_part_Certificate['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Certificate['FLE'], errors='coerce')

enrolment_Certificate_UHC = df_enrolment_filtered_full_part_Certificate['PEOPLE_CODE_ID'].nunique()
enrolment_Certificate_FLE = df_enrolment_filtered_full_part_Certificate['FLE'].sum()

#Determine the Diploma UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Diploma = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['CREDENTIAL_TYPE'].str.replace(' ','').str.upper().isin(['DIPLOMA']))
]
df_enrolment_filtered_full_part_Diploma['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Diploma['FLE'], errors='coerce')

enrolment_Diploma_UHC = df_enrolment_filtered_full_part_Diploma['PEOPLE_CODE_ID'].nunique()
enrolment_Diploma_FLE = df_enrolment_filtered_full_part_Diploma['FLE'].sum()

#Determine the Non-Credential UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Non_Credential = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['CREDENTIAL_TYPE'].str.replace(' ','').str.upper().isin(['NON-CREDENTIAL','NONCREDENTIAL']))
]
df_enrolment_filtered_full_part_Non_Credential['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Non_Credential['FLE'], errors='coerce')

enrolment_Non_Credential_UHC = df_enrolment_filtered_full_part_Non_Credential['PEOPLE_CODE_ID'].nunique()
enrolment_Non_Credential_FLE = df_enrolment_filtered_full_part_Non_Credential['FLE'].sum()

#Determine the Fall term UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Fall = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year))
]
df_enrolment_filtered_full_part_Fall['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Fall['FLE'], errors='coerce')

enrolment_Fall_UHC = df_enrolment_filtered_full_part_Fall['PEOPLE_CODE_ID'].nunique()
enrolment_Fall_FLE = df_enrolment_filtered_full_part_Fall['FLE'].sum()

#Determine the Winter term UHC and FLE for the current academic year
df_enrolment_filtered_full_part_Winter = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year))
]
df_enrolment_filtered_full_part_Winter['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Winter['FLE'], errors='coerce')

enrolment_Winter_UHC = df_enrolment_filtered_full_part_Winter['PEOPLE_CODE_ID'].nunique()
enrolment_Winter_FLE = df_enrolment_filtered_full_part_Winter['FLE'].sum()

#Calculations for FLE for the current academic year
Enrolment_total_FLE_projection = float(enrolment_projection_map["FLE Projection All Students"])

enrolment_Total_FLE

try:
    if float(enrolment_Total_FLE) > float(Enrolment_total_FLE_projection):
        Enrolment_FLE_percentage_number = percent_difference_value_calculator_round_1(enrolment_Total_FLE, Enrolment_total_FLE_projection)
    else:
        Enrolment_FLE_percentage_number = division_calculator(enrolment_Total_FLE, Enrolment_total_FLE_projection)
except Exception:
    Enrolment_FLE_percentage_number = "N/A"

try:
    Enrolment_FLE_percentage_number_absolute = np.abs(Enrolment_FLE_percentage_number)
except (TypeError, ValueError):
    Enrolment_FLE_percentage_number_absolute = "N/A"

try:
    if str(Enrolment_FLE_percentage_number_absolute).strip().startswith('8'):
        a_an_1 = "an"
    else:
        a_an_1 = "a"

except Exception: 
    a_an_1 = "a"

try:
    if float(enrolment_Total_FLE) > float(Enrolment_total_FLE_projection) >= 0:
        surpass_NULL_1 = "surpass "
    else:
        surpass_NULL_1 = ""

except (TypeError, ValueError): 
    surpass_NULL_1 = "FAILED TO CALCULATE"

#Calculations for the Fall term part-time paragraph, for the current academic year
Enrolment_term_year_1 = 'Fall' + ' ' + str(enrolment_start_year)

Enrolment_term_year_1_part_time_HC_projection = int(enrolment_projection_map["Fall Part-time Headcount Projection All Students (Excluding CML/LINC/APPR)"])

df_enrolment_filtered_part_Fall = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_1_part_time_UHC_per_curriculum = df_enrolment_filtered_part_Fall.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_1_part_time_HC = Enrolment_term_year_1_part_time_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_1_part_time_HC) > int(Enrolment_term_year_1_part_time_HC_projection):
        Enrolment_term_year_1_part_time_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_1_part_time_HC, Enrolment_term_year_1_part_time_HC_projection)
    else:
        Enrolment_term_year_1_part_time_HC_percent_change_number = division_calculator(Enrolment_term_year_1_part_time_HC, Enrolment_term_year_1_part_time_HC_projection)
except Exception:
    Enrolment_term_year_1_part_time_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_1_part_time_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_1_part_time_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_1_part_time_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_1_part_time_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_2 = "an"
    else:
        a_an_2 = "a"

except Exception: 
    a_an_2 = "a"

try:
    if int(Enrolment_term_year_1_part_time_HC) > int(Enrolment_term_year_1_part_time_HC_projection) >= 0:
        surpass_NULL_2 = "surpass "
    else:
        surpass_NULL_2 = ""

except (TypeError, ValueError): 
    surpass_NULL_2 = "FAILED TO CALCULATE"

#Calculations for the Fall term full-time paragraph, for the current academic year
Enrolment_term_year_1 = 'Fall' + ' ' + str(enrolment_start_year)

Enrolment_term_year_1_full_time_HC_projection = int(enrolment_projection_map["Fall Full-time Headcount Projection All Students (Excluding CML/LINC/APPR)"])

df_enrolment_filtered_full_Fall = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_1_full_time_UHC_per_curriculum = df_enrolment_filtered_full_Fall.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_1_full_time_HC = Enrolment_term_year_1_full_time_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_1_full_time_HC) > int(Enrolment_term_year_1_full_time_HC_projection):
        Enrolment_term_year_1_full_time_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_1_full_time_HC, Enrolment_term_year_1_full_time_HC_projection)
    else:
        Enrolment_term_year_1_full_time_HC_percent_change_number = division_calculator(Enrolment_term_year_1_full_time_HC, Enrolment_term_year_1_full_time_HC_projection)
except Exception:
    Enrolment_term_year_1_full_time_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_1_full_time_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_1_full_time_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_1_full_time_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_1_full_time_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_3 = "an"
    else:
        a_an_3 = "a"

except Exception: 
    a_an_3 = "a"

try:
    if int(Enrolment_term_year_1_full_time_HC) > int(Enrolment_term_year_1_full_time_HC_projection) >= 0:
        surpass_NULL_3 = "surpass "
    else:
        surpass_NULL_3 = ""

except (TypeError, ValueError): 
    surpass_NULL_3 = "FAILED TO CALCULATE"

#Calculations for the Winter term part-time paragraph, for the current academic year
Enrolment_term_year_2 = 'Winter' + ' ' + str(enrolment_end_year)

Enrolment_term_year_2_part_time_HC_projection = int(enrolment_projection_map["Winter Part-time Headcount Projection All Students (Excluding CML/LINC/APPR)"])

df_enrolment_filtered_part_Winter = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_2_part_time_UHC_per_curriculum = df_enrolment_filtered_part_Winter.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_2_part_time_HC = Enrolment_term_year_2_part_time_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_2_part_time_HC) > int(Enrolment_term_year_2_part_time_HC_projection):
        Enrolment_term_year_2_part_time_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_2_part_time_HC, Enrolment_term_year_2_part_time_HC_projection)
    else:
        Enrolment_term_year_2_part_time_HC_percent_change_number = division_calculator(Enrolment_term_year_2_part_time_HC, Enrolment_term_year_2_part_time_HC_projection)
except Exception:
    Enrolment_term_year_2_part_time_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_2_part_time_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_2_part_time_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_2_part_time_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_2_part_time_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_4 = "an"
    else:
        a_an_4 = "a"

except Exception: 
    a_an_4 = "a"

try:
    if int(Enrolment_term_year_2_part_time_HC) > int(Enrolment_term_year_2_part_time_HC_projection):
        surpass_NULL_4 = "surpass "
    else:
        surpass_NULL_4 = ""

except (TypeError, ValueError): 
    surpass_NULL_4 = "FAILED TO CALCULATE"

#Calculations for the Winter term full-time paragraph, for the current academic year
Enrolment_term_year_2 = 'Winter' + ' ' + str(enrolment_end_year)

Enrolment_term_year_2_full_time_HC_projection = int(enrolment_projection_map["Winter Full-time Headcount Projection All Students (Excluding CML/LINC/APPR)"])

df_enrolment_filtered_full_Winter = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_2_full_time_UHC_per_curriculum = df_enrolment_filtered_full_Winter.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_2_full_time_HC = Enrolment_term_year_2_full_time_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_2_full_time_HC) > int(Enrolment_term_year_2_full_time_HC_projection):
        Enrolment_term_year_2_full_time_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_2_full_time_HC, Enrolment_term_year_2_full_time_HC_projection)
    else:
        Enrolment_term_year_2_full_time_HC_percent_change_number = division_calculator(Enrolment_term_year_2_full_time_HC, Enrolment_term_year_2_full_time_HC_projection)
except Exception:
    Enrolment_term_year_2_full_time_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_2_full_time_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_2_full_time_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_2_full_time_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_2_full_time_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_5 = "an"
    else:
        a_an_5 = "a"

except Exception: 
    a_an_5 = "a"

try:
    if int(Enrolment_term_year_2_full_time_HC) > int(Enrolment_term_year_2_full_time_HC_projection):
        surpass_NULL_5 = "surpass "
    else:
        surpass_NULL_5 = ""

except (TypeError, ValueError): 
    surpass_NULL_5 = "FAILED TO CALCULATE"

#Term year for LINC last intake
Enrolment_LINC_last_intake_term_year = "May " + str(enrolment_end_year)

#Calculations for the Summer term in the "Enrolments Projections Progress by Unique Headcount" table
df_enrolment_filtered_full_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['SUMMER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR = df_enrolment_filtered_full_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR = enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR), int(enrolment_projection_map["Summer Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['SUMMER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR = df_enrolment_filtered_part_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR = enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR), int(enrolment_projection_map["Summer Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_full_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['SUMMER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_International_Summer_exclude_CML_LINC_APPR = df_enrolment_filtered_full_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_International_Summer_exclude_CML_LINC_APPR = enrolment_full_International_Summer_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_International_Summer_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_International_Summer_exclude_CML_LINC_APPR), int(enrolment_projection_map["Summer Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_International_Summer_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['SUMMER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_International_Summer_exclude_CML_LINC_APPR = df_enrolment_filtered_part_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_International_Summer_exclude_CML_LINC_APPR = enrolment_part_International_Summer_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_International_Summer_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_International_Summer_exclude_CML_LINC_APPR), int(enrolment_projection_map["Summer Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_International_Summer_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

#Calculations for the Fall term in the "Enrolments Projections Progress by Unique Headcount" table
df_enrolment_filtered_full_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR = df_enrolment_filtered_full_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR = enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR), int(enrolment_projection_map["Fall Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR = df_enrolment_filtered_part_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR = enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR), int(enrolment_projection_map["Fall Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_full_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_International_Fall_exclude_CML_LINC_APPR = df_enrolment_filtered_full_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_International_Fall_exclude_CML_LINC_APPR = enrolment_full_International_Fall_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_International_Fall_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_International_Fall_exclude_CML_LINC_APPR), int(enrolment_projection_map["Fall Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_International_Fall_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_International_Fall_exclude_CML_LINC_APPR = df_enrolment_filtered_part_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_International_Fall_exclude_CML_LINC_APPR = enrolment_part_International_Fall_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_International_Fall_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_International_Fall_exclude_CML_LINC_APPR), int(enrolment_projection_map["Fall Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_International_Fall_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

#Calculations for the Winter term in the "Enrolments Projections Progress by Unique Headcount" table
df_enrolment_filtered_full_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR = df_enrolment_filtered_full_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR = enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR), int(enrolment_projection_map["Winter Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_Domestic = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['CANADIAN', 'PERMANENTRESIDENT', 'REFUGEE'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR = df_enrolment_filtered_part_Domestic.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR = enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR), int(enrolment_projection_map["Winter Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_full_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_full_International_Winter_exclude_CML_LINC_APPR = df_enrolment_filtered_full_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_full_International_Winter_exclude_CML_LINC_APPR = enrolment_full_International_Winter_exclude_CML_LINC_APPR.sum()

try:
    enrolment_full_International_Winter_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_full_International_Winter_exclude_CML_LINC_APPR), int(enrolment_projection_map["Winter Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_full_International_Winter_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

df_enrolment_filtered_part_International = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['PART'])) &
    (df_enrolment['LEGAL'].str.replace(' ','').str.upper().isin(['STUDENTVISA', 'OTHERVISA'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (~df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['POWCM3', 'POWCM4', 'LINC13', 'ACARP', 'AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

enrolment_part_International_Winter_exclude_CML_LINC_APPR = df_enrolment_filtered_part_International.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
enrolment_part_International_Winter_exclude_CML_LINC_APPR = enrolment_part_International_Winter_exclude_CML_LINC_APPR.sum()

try:
    enrolment_part_International_Winter_exclude_CML_LINC_APPR_percentage = f"{division_calculator(int(enrolment_part_International_Winter_exclude_CML_LINC_APPR), int(enrolment_projection_map["Winter Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"]))}%"
except Exception:
    enrolment_part_International_Winter_exclude_CML_LINC_APPR_percentage = "FAILED TO CALCULATE"

#Calculations for the Fall term Apprenticeship paragraph, for the current academic year
Enrolment_term_year_1 = 'Fall' + ' ' + str(enrolment_start_year)

Enrolment_term_year_1_Apprenticeship_HC_projection = int(enrolment_projection_map_Fall_Apprenticeship["Total"])

df_enrolment_filtered_full_Fall = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_1_Apprenticeship_UHC_per_curriculum = df_enrolment_filtered_full_Fall.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_1_Apprenticeship_HC = Enrolment_term_year_1_Apprenticeship_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_1_Apprenticeship_HC) > int(Enrolment_term_year_1_Apprenticeship_HC_projection):
        Enrolment_term_year_1_Apprenticeship_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_1_Apprenticeship_HC, Enrolment_term_year_1_Apprenticeship_HC_projection)
    else:
        Enrolment_term_year_1_Apprenticeship_HC_percent_change_number = division_calculator(Enrolment_term_year_1_Apprenticeship_HC, Enrolment_term_year_1_Apprenticeship_HC_projection)
except Exception:
    Enrolment_term_year_1_Apprenticeship_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_1_Apprenticeship_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_1_Apprenticeship_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_1_Apprenticeship_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_1_Apprenticeship_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_6 = "an"
    else:
        a_an_6 = "a"

except Exception: 
    a_an_6 = "a"

try:
    if int(Enrolment_term_year_1_Apprenticeship_HC) > int(Enrolment_term_year_1_Apprenticeship_HC_projection) >= 0:
        surpass_NULL_6 = "surpass "
    else:
        surpass_NULL_6 = ""

except (TypeError, ValueError): 
    surpass_NULL_6 = "FAILED TO CALCULATE"

#Calculations for the Winter term Apprenticeship paragraph, for the current academic year
Enrolment_term_year_2 = 'Winter' + ' ' + str(enrolment_end_year)

Enrolment_term_year_2_Apprenticeship_HC_projection = int(enrolment_projection_map_Winter_Apprenticeship["Total"])

df_enrolment_filtered_full_Winter = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AELEC', 'AHD', 'AMILL', 'ASTEAM', 'AWELD']))
]

Enrolment_term_year_2_Apprenticeship_UHC_per_curriculum = df_enrolment_filtered_full_Winter.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()
Enrolment_term_year_2_Apprenticeship_HC = Enrolment_term_year_2_Apprenticeship_UHC_per_curriculum.sum()

try:
    if int(Enrolment_term_year_2_Apprenticeship_HC) > int(Enrolment_term_year_2_Apprenticeship_HC_projection):
        Enrolment_term_year_2_Apprenticeship_HC_percent_change_number = percent_difference_value_calculator_round_1(Enrolment_term_year_2_Apprenticeship_HC, Enrolment_term_year_2_Apprenticeship_HC_projection)
    else:
        Enrolment_term_year_2_Apprenticeship_HC_percent_change_number = division_calculator(Enrolment_term_year_2_Apprenticeship_HC, Enrolment_term_year_2_Apprenticeship_HC_projection)
except Exception:
    Enrolment_term_year_2_Apprenticeship_HC_percent_change_number = "N/A"

try:
    Enrolment_term_year_2_Apprenticeship_HC_percent_change_number_absolute = np.abs(Enrolment_term_year_2_Apprenticeship_HC_percent_change_number)
except (TypeError, ValueError):
    Enrolment_term_year_2_Apprenticeship_HC_percent_change_number_absolute = "N/A"

try:
    if str(Enrolment_term_year_2_Apprenticeship_HC_percent_change_number_absolute).strip().startswith('8'):
        a_an_7 = "an"
    else:
        a_an_7 = "a"

except Exception: 
    a_an_7 = "a"

try:
    if int(Enrolment_term_year_2_Apprenticeship_HC) > int(Enrolment_term_year_2_Apprenticeship_HC_projection) >= 0:
        surpass_NULL_7 = "surpass "
    else:
        surpass_NULL_7 = ""

except (TypeError, ValueError): 
    surpass_NULL_7 = "FAILED TO CALCULATE"

#Calculations for top 5 International programs
df_enrolment_filtered_full_part_International_copy = df_enrolment_filtered_full_part_International.copy()

# Normalize curriculum names (handle stray spaces)
df_enrolment_filtered_full_part_International_copy['CURRICULUM'] = (
    df_enrolment_filtered_full_part_International_copy['CURRICULUM']
    .str.strip()  # remove leading/trailing spaces
    .replace({'BSCACN': 'BSCN', 'BSCAFT': 'BSCN', 'BSCAEE': 'BSCN'})
)

# Count distinct students by curriculum
df_grouped = (
    df_enrolment_filtered_full_part_International_copy
    .groupby('CURRICULUM')['PEOPLE_CODE_ID']
    .nunique()
    .sort_values(ascending=False)
)

# Get top 5 (or fewer)
top_programs = list(df_grouped.index[:5])

# Create a safe accessor that returns '' if the rank doesn’t exist
def get_rank_name(rank_idx):
    try:
        code = top_programs[rank_idx]
        return active_program_map.get(code, "")  # safe lookup
    except IndexError:
        return ""

# Assign rank variables safely
rank_1_International_program_name = get_rank_name(0)
rank_2_International_program_name = get_rank_name(1)
rank_3_International_program_name = get_rank_name(2)
rank_4_International_program_name = get_rank_name(3)
rank_5_International_program_name = get_rank_name(4)

rank_list = [rank_2_International_program_name,rank_3_International_program_name,rank_4_International_program_name,rank_5_International_program_name]

# Filter out blanks
valid_ranks = [r for r in rank_list if r.strip() != '']

# Construct the sentence
if valid_ranks:
    # Join with commas and 'and' before the last item
    if len(valid_ranks) == 1:
        rank_sentence = f"Increased numbers are visible in {valid_ranks[0]}."
    elif len(valid_ranks) == 2:
        rank_sentence = f"Increased numbers are visible in {valid_ranks[0]} and {valid_ranks[1]}."
    else:
        rank_sentence = f"Increased numbers are visible in {', '.join(valid_ranks[:-1])}, and {valid_ranks[-1]}."
else:
    rank_sentence = ''

#International FLE vs. Total FLE

enrolment_International_FLE_percentage = division_calculator(enrolment_International_FLE, enrolment_Total_FLE)

#Calculations for top Indigenous programs
df_enrolment_filtered_full_part_Indigenous_copy = df_enrolment_filtered_full_part_Indigenous.copy()

df_enrolment_filtered_full_part_Indigenous_copy['CURRICULUM'] = \
    df_enrolment_filtered_full_part_Indigenous_copy['CURRICULUM'].replace({
        'BSCACN': 'BSCN',
        'BSCAFT': 'BSCN',
        'BSCAEE' : 'BSCN'
    })

df_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM = df_enrolment_filtered_full_part_Indigenous_copy.groupby('CURRICULUM')['PEOPLE_CODE_ID'].nunique()


top_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM = df_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM.sort_values(ascending=False)
rank_1_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM = top_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM.index[0]

rank_1_Indigenous_program_name = active_program_map[rank_1_enrolment_filtered_full_part_Indigenous_copy_group_by_CURRICULUM]

#Indigenous FLE vs. Total FLE

enrolment_Indigenous_FLE_percentage = division_calculator(enrolment_Indigenous_FLE, enrolment_Total_FLE)

C:\Users\Bill.Guo\AppData\Local\Temp\ipykernel_28140\3805658606.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enrolment_filtered_full_part_Total['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_Total['FLE'], errors='coerce')
C:\Users\Bill.Guo\AppData\Local\Temp\ipykernel_28140\3805658606.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enrolment_filtered_full_part_International['FLE'] = pd.to_numeric(df_enrolment_filtered_full_part_International['FLE'], errors='coerce')
C:\Users\Bill.

In [23]:
#Special Calculations for Apprenticeship Paragraphs

#List for storing Apprenticeship programs that achieved or surpassed projection in Fall
enrolment_Aprrenticeship_Fall_achieved_surpassed = ['None']

#Fall Apprenticeship Electrician (AELEC)
df_enrolment_Fall_Apprenticeship_AELEC = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AELEC']))
]

df_groupcount = df_enrolment_Fall_Apprenticeship_AELEC.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Fall_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Fall_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AELEC rows from df_matches
df_AELEC = df_matches[df_matches['Curriculum'] == 'AELEC']

if df_AELEC.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AELEC['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AELEC"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Fall_achieved_surpassed:
            enrolment_Aprrenticeship_Fall_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Fall_achieved_surpassed.append(output_str)

#Fall Apprenticeship Heavy Equipment Technician (AHD)
df_enrolment_Fall_Apprenticeship_AHD = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AHD']))
]

df_groupcount = df_enrolment_Fall_Apprenticeship_AHD.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Fall_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Fall_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AHD rows from df_matches
df_AHD = df_matches[df_matches['Curriculum'] == 'AHD']

if df_AHD.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AHD['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AHD"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Fall_achieved_surpassed:
            enrolment_Aprrenticeship_Fall_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Fall_achieved_surpassed.append(output_str)

#Fall Apprenticeship Industrial Mechanic (Millwright) (AMILL)
df_enrolment_Fall_Apprenticeship_AMILL = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AMILL']))
]

df_groupcount = df_enrolment_Fall_Apprenticeship_AMILL.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Fall_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Fall_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AMILL rows from df_matches
df_AMILL = df_matches[df_matches['Curriculum'] == 'AMILL']

if df_AMILL.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AMILL['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AMILL"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Fall_achieved_surpassed:
            enrolment_Aprrenticeship_Fall_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Fall_achieved_surpassed.append(output_str)

#Fall Apprenticeship Steamfitter - Pipefitter (ASTEAM)
df_enrolment_Fall_Apprenticeship_ASTEAM = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['ASTEAM']))
]

df_groupcount = df_enrolment_Fall_Apprenticeship_ASTEAM.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Fall_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Fall_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just ASTEAM rows from df_matches
df_ASTEAM = df_matches[df_matches['Curriculum'] == 'ASTEAM']

if df_ASTEAM.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_ASTEAM['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["ASTEAM"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Fall_achieved_surpassed:
            enrolment_Aprrenticeship_Fall_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Fall_achieved_surpassed.append(output_str)

#Fall Apprenticeship Welder (AWELD)
df_enrolment_Fall_Apprenticeship_AWELD = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['FALL'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_start_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AWELD']))
]

df_groupcount = df_enrolment_Fall_Apprenticeship_AWELD.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Fall_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Fall_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AWELD rows from df_matches
df_AWELD = df_matches[df_matches['Curriculum'] == 'AWELD']

if df_AWELD.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AWELD['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AWELD"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Fall_achieved_surpassed:
            enrolment_Aprrenticeship_Fall_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Fall_achieved_surpassed.append(output_str)







#List for storing Apprenticeship programs that achieved or surpassed projection in Winter
enrolment_Aprrenticeship_Winter_achieved_surpassed = ['None']

#Winter Apprenticeship Electrician (AELEC)
df_enrolment_Winter_Apprenticeship_AELEC = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AELEC']))
]

df_groupcount = df_enrolment_Winter_Apprenticeship_AELEC.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Winter_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Winter_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AELEC rows from df_matches
df_AELEC = df_matches[df_matches['Curriculum'] == 'AELEC']

if df_AELEC.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AELEC['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AELEC"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Winter_achieved_surpassed:
            enrolment_Aprrenticeship_Winter_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Winter_achieved_surpassed.append(output_str)

#Winter Apprenticeship Heavy Equipment Technician (AHD)
df_enrolment_Winter_Apprenticeship_AHD = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AHD']))
]

df_groupcount = df_enrolment_Winter_Apprenticeship_AHD.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Winter_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Winter_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AHD rows from df_matches
df_AHD = df_matches[df_matches['Curriculum'] == 'AHD']

if df_AHD.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AHD['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AHD"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Winter_achieved_surpassed:
            enrolment_Aprrenticeship_Winter_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Winter_achieved_surpassed.append(output_str)

#Winter Apprenticeship Industrial Mechanic (Millwright) (AMILL)
df_enrolment_Winter_Apprenticeship_AMILL = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AMILL']))
]

df_groupcount = df_enrolment_Winter_Apprenticeship_AMILL.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Winter_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Winter_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AMILL rows from df_matches
df_AMILL = df_matches[df_matches['Curriculum'] == 'AMILL']

if df_AMILL.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AMILL['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AMILL"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Winter_achieved_surpassed:
            enrolment_Aprrenticeship_Winter_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Winter_achieved_surpassed.append(output_str)

#Winter Apprenticeship Steamfitter - Pipefitter (ASTEAM)
df_enrolment_Winter_Apprenticeship_ASTEAM = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['ASTEAM']))
]

df_groupcount = df_enrolment_Winter_Apprenticeship_ASTEAM.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Winter_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Winter_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just ASTEAM rows from df_matches
df_ASTEAM = df_matches[df_matches['Curriculum'] == 'ASTEAM']

if df_ASTEAM.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_ASTEAM['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["ASTEAM"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Winter_achieved_surpassed:
            enrolment_Aprrenticeship_Winter_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Winter_achieved_surpassed.append(output_str)

#Winter Apprenticeship Welder (AWELD)
df_enrolment_Winter_Apprenticeship_AWELD = df_enrolment[
    (df_enrolment['FULL_PART'].str.replace(' ','').str.upper().isin(['FULL', 'PART'])) &
    (df_enrolment['ACADEMIC_TERM'].str.replace(' ','').str.upper().isin(['WINTER'])) &
    (df_enrolment['ACADEMIC_YEAR'].astype(str).str.strip() == str(enrolment_end_year)) &
    (df_enrolment['CURRICULUM'].str.replace(' ','').str.upper().isin(['AWELD']))
]

df_groupcount = df_enrolment_Winter_Apprenticeship_AWELD.groupby(['CURRICULUM', 'CLASS_LEVEL'])['PEOPLE_CODE_ID'].count()

# Collect matching items (met or exceeded projections)
matches = []

for (curriculum, class_level), count in df_groupcount.items():
    # Check if curriculum and class level exist in the projection map
    if curriculum in enrolment_projection_map_Winter_Apprenticeship:
        class_level_thresholds = enrolment_projection_map_Winter_Apprenticeship[curriculum]
        
        # Class levels are strings in the map, so convert if needed
        class_level_str = str(class_level)
        if class_level_str in class_level_thresholds:
            threshold = class_level_thresholds[class_level_str]
            if threshold > 0 and count >= threshold:
                matches.append((curriculum, class_level, count, threshold))

# Convert results to a readable DataFrame
df_matches = pd.DataFrame(matches, columns=["Curriculum", "Class Level", "Enrolment Count", "Threshold"])

# Filter just AWELD rows from df_matches
df_AWELD = df_matches[df_matches['Curriculum'] == 'AWELD']

if df_AWELD.empty:
    output_str = ""  # Nothing matched
else:
    # Extract and sort the class levels (convert to strings to match the map)
    class_levels = df_AWELD['Class Level'].astype(str).tolist()

    # Translate class levels to English using the map
    translated_levels = [class_level_map[level] for level in class_levels]

    # Sort by original class level number for logical order (1, 2, 4)
    sorted_levels = sorted(translated_levels, key=lambda x: list(class_level_map.values()).index(x))

    # Create a natural language string (e.g., "First year, Second year, and Fourth year")
    if len(sorted_levels) == 1:
        levels_str = sorted_levels[0]
    elif len(sorted_levels) == 2:
        levels_str = f"{sorted_levels[0]} and {sorted_levels[1]}"
    else:
        levels_str = ", ".join(sorted_levels[:-1]) + f", and {sorted_levels[-1]}"

    # Get the English program name
    program_name = active_program_map["AWELD"]

    # Combine into final string
    output_str = f"{program_name} - {levels_str}"

    # Append output_str to list if it is not empty
    if output_str != '':
        if 'None' in enrolment_Aprrenticeship_Winter_achieved_surpassed:
            enrolment_Aprrenticeship_Winter_achieved_surpassed.remove('None')
        enrolment_Aprrenticeship_Winter_achieved_surpassed.append(output_str)



In [24]:
#Convert Fall Apprenticeship list to bullet points
hollow_bullet = '⚬'  # You can also try '○' or '⚬' if you prefer
space = '\u00A0' * 3  # 3 non-breaking spaces
enrolment_Aprrenticeship_Fall_achieved_surpassed_bullet_points = '\n'.join([f'{hollow_bullet}{space}{item}' for item in enrolment_Aprrenticeship_Fall_achieved_surpassed])

#Convert Winter Apprenticeship list to bullet points
hollow_bullet = '⚬'  # You can also try '○' or '⚬' if you prefer
space = '\u00A0' * 3  # 3 non-breaking spaces
enrolment_Aprrenticeship_Winter_achieved_surpassed_bullet_points = '\n'.join([f'{hollow_bullet}{space}{item}' for item in enrolment_Aprrenticeship_Winter_achieved_surpassed])

In [25]:
#Previous Academic Year Enrolment Data and Calculations

# Load the workbook
wb = load_workbook(enrolment_file_previous_year_summary, data_only=True)
ws = wb['Previous Year Enrolment Table']

# Extract values based on cell location in Excel
value_C3 = ws['C3'].value

C3_date_match = re.search(r'\d{4}-\d{2}-\d{2}', value_C3)
C3_year_match = re.search(r'\(\d{4}-\d{2}\)', value_C3)

previous_year_Extraction_Date = C3_date_match.group() if C3_date_match else None
previous_year_enrolment_academic_year = C3_year_match.group().strip('()') if C3_year_match else None

previous_enrolment_Total_UHC = int(float(ws['C6'].value.strip())) if isinstance(ws['C6'].value, str) else int(float(ws['C6'].value))
previous_enrolment_International_UHC = int(float(ws['C7'].value.strip())) if isinstance(ws['C7'].value, str) else int(float(ws['C7'].value))
previous_enrolment_Domestic_UHC = int(float(ws['C8'].value.strip())) if isinstance(ws['C8'].value, str) else int(float(ws['C8'].value))
previous_enrolment_Indigenous_UHC = int(float(ws['C9'].value.strip())) if isinstance(ws['C9'].value, str) else int(float(ws['C9'].value))
previous_enrolment_Apprenticeship_UHC = int(float(ws['C10'].value.strip())) if isinstance(ws['C10'].value, str) else int(float(ws['C10'].value))
previous_enrolment_Certificate_UHC = int(float(ws['C11'].value.strip())) if isinstance(ws['C11'].value, str) else int(float(ws['C11'].value))
previous_enrolment_Diploma_UHC = int(float(ws['C12'].value.strip())) if isinstance(ws['C12'].value, str) else int(float(ws['C12'].value))
previous_enrolment_Non_Credential_UHC = int(float(ws['C13'].value.strip())) if isinstance(ws['C13'].value, str) else int(float(ws['C13'].value))
previous_enrolment_Fall_UHC = int(float(ws['C14'].value.strip())) if isinstance(ws['C14'].value, str) else int(float(ws['C14'].value))
previous_enrolment_Winter_UHC = int(float(ws['C15'].value.strip())) if isinstance(ws['C15'].value, str) else int(float(ws['C15'].value))
previous_enrolment_Total_FLE = float(ws['D6'].value.strip()) if isinstance(ws['D6'].value, str) else float(ws['D6'].value)
previous_enrolment_International_FLE = float(ws['D7'].value.strip()) if isinstance(ws['D7'].value, str) else float(ws['D7'].value)
previous_enrolment_Domestic_FLE = float(ws['D8'].value.strip()) if isinstance(ws['D8'].value, str) else float(ws['D8'].value)
previous_enrolment_Indigenous_FLE = float(ws['D9'].value.strip()) if isinstance(ws['D9'].value, str) else float(ws['D9'].value)
previous_enrolment_Apprenticeship_FLE = float(ws['D10'].value.strip()) if isinstance(ws['D10'].value, str) else float(ws['D10'].value)
previous_enrolment_Certificate_FLE = float(ws['D11'].value.strip()) if isinstance(ws['D11'].value, str) else float(ws['D11'].value)
previous_enrolment_Diploma_FLE = float(ws['D12'].value.strip()) if isinstance(ws['D12'].value, str) else float(ws['D12'].value)
previous_enrolment_Non_Credential_FLE = float(ws['D13'].value.strip()) if isinstance(ws['D13'].value, str) else float(ws['D13'].value)
previous_enrolment_Fall_FLE = float(ws['D14'].value.strip()) if isinstance(ws['D14'].value, str) else float(ws['D14'].value)
previous_enrolment_Winter_FLE = float(ws['D15'].value.strip()) if isinstance(ws['D15'].value, str) else float(ws['D15'].value)

In [26]:
#Other Enrolment Calculations

#Percentage number calculations for enrolment UHC values
enrolment_Total_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Total_UHC, previous_enrolment_Total_UHC)}%"
enrolment_International_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_International_UHC, previous_enrolment_International_UHC)}%"
enrolment_Domestic_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Domestic_UHC, previous_enrolment_Domestic_UHC)}%"
enrolment_Indigenous_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Indigenous_UHC, previous_enrolment_Indigenous_UHC)}%"
enrolment_Apprenticeship_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Apprenticeship_UHC, previous_enrolment_Apprenticeship_UHC)}%"
enrolment_Certificate_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Certificate_UHC, previous_enrolment_Certificate_UHC)}%"
enrolment_Diploma_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Diploma_UHC, previous_enrolment_Diploma_UHC)}%"
enrolment_Non_Credential_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Non_Credential_UHC, previous_enrolment_Non_Credential_UHC)}%"
enrolment_Fall_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Fall_UHC, previous_enrolment_Fall_UHC)}%"
enrolment_Winter_percent_change_UHC = f"{percent_difference_value_calculator_round_1(enrolment_Winter_UHC, previous_enrolment_Winter_UHC)}%"

#Percentage number calculations for enrolment FLE values
enrolment_Total_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Total_FLE, previous_enrolment_Total_FLE)}%"
enrolment_International_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_International_FLE, previous_enrolment_International_FLE)}%"
enrolment_Domestic_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Domestic_FLE, previous_enrolment_Domestic_FLE)}%"
enrolment_Indigenous_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Indigenous_FLE, previous_enrolment_Indigenous_FLE)}%"
enrolment_Apprenticeship_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Apprenticeship_FLE, previous_enrolment_Apprenticeship_FLE)}%"
enrolment_Certificate_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Certificate_FLE, previous_enrolment_Certificate_FLE)}%"
enrolment_Diploma_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Diploma_FLE, previous_enrolment_Diploma_FLE)}%"
enrolment_Non_Credential_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Non_Credential_FLE, previous_enrolment_Non_Credential_FLE)}%"
enrolment_Fall_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Fall_FLE, previous_enrolment_Fall_FLE)}%"
enrolment_Winter_percent_change_FLE = f"{percent_difference_value_calculator_round_1(enrolment_Winter_FLE, previous_enrolment_Winter_FLE)}%"




In [ ]:
# Fill in Briefing Note Word Document
# Load the Word template
doc = DocxTemplate("Briefing Note - KC Applications and Enrolments KPI Update_TEMPLATE.docx")

# Define your values
raw_data = {
    #Briefing Note Title Headers Map
    "Prepared_for": variable_Prepared_for.replace("&", "&amp;"),
    "Title_1": variable_Title1,
    "Date_prepared": variable_Date_prepared,
    "Prepared_by": variable_Prepared_by,
    "Title_2": variable_Title2,
    "Department": variable_Department,
    "Extraction_date": variable_Extraction_date,

    #Briefing Note Applications Map
    "Applications_header_1": term_1_term_year_current,
    "Applications_header_2": term_1_term_year_minus_one,
    "Applications_header_3": term_2_term_year_current,
    "Applications_header_4": term_2_term_year_minus_one,
    "Applications_header_5": term_3_term_year_current,
    "Applications_header_6": term_3_term_year_minus_one,
    "Applications_box_1": term_1_details_total_applications,
    "Applications_box_2": term_1_details_international_applications,
    "Applications_box_3": term_1_details_domestic_applications,
    "Applications_box_4": term_1_year_minus_one_Total_Apps_All_Applicants,
    "Applications_box_5": term_1_year_minus_one_Total_Apps_International_Applicants,
    "Applications_box_6": term_1_year_minus_one_Total_Apps_Domestic_Applicants,
    "Applications_box_7": "N/A" if term_1_total_applications_percent_change == "N/A%" else term_1_total_applications_percent_change,
    "Applications_box_8": "N/A" if term_1_international_applications_percent_change == "N/A%" else term_1_international_applications_percent_change,
    "Applications_box_9": "N/A" if term_1_domestic_applications_percent_change == "N/A%" else term_1_domestic_applications_percent_change,
    "Applications_box_10": term_2_details_total_applications,
    "Applications_box_11": term_2_details_international_applications,
    "Applications_box_12": term_2_details_domestic_applications,
    "Applications_box_13": term_2_year_minus_one_Total_Apps_All_Applicants,
    "Applications_box_14": term_2_year_minus_one_Total_Apps_International_Applicants,
    "Applications_box_15": term_2_year_minus_one_Total_Apps_Domestic_Applicants,
    "Applications_box_16": "N/A" if term_2_total_applications_percent_change == "N/A%" else term_2_total_applications_percent_change,
    "Applications_box_17": "N/A" if term_2_international_applications_percent_change == "N/A%" else term_2_international_applications_percent_change,
    "Applications_box_18": "N/A" if term_2_domestic_applications_percent_change == "N/A%" else term_2_domestic_applications_percent_change,
    "Applications_box_19": term_3_details_total_applications,
    "Applications_box_20": term_3_details_international_applications,
    "Applications_box_21": term_3_details_domestic_applications,
    "Applications_box_22": term_3_year_minus_one_Total_Apps_All_Applicants,
    "Applications_box_23": term_3_year_minus_one_Total_Apps_International_Applicants,
    "Applications_box_24": term_3_year_minus_one_Total_Apps_Domestic_Applicants,
    "Applications_box_25": "N/A" if term_3_total_applications_percent_change == "N/A%" else term_3_total_applications_percent_change,
    "Applications_box_26": "N/A" if term_3_international_applications_percent_change == "N/A%" else term_3_international_applications_percent_change,
    "Applications_box_27": "N/A" if term_3_domestic_applications_percent_change == "N/A%" else term_3_domestic_applications_percent_change,
    "term_1_details_total_unique_applicants": term_1_details_total_unique_applicants,
    "term_1_details_total_applications": term_1_details_total_applications,
    "term_1_total_applications_percent_change_number_unique_absolute": term_1_total_applications_percent_change_number_unique_absolute,
    "higher_lower_1": higher_lower_1,
    "term_1_details_accepted_applications": term_1_details_accepted_applications,
    "term_1_acceptance_percentage_number": term_1_acceptance_percentage_number,
    "term_1_details_international_applications": term_1_details_international_applications,
    "term_1_international_percentage_number": term_1_international_percentage_number,
    "term_2_details_total_unique_applicants": term_2_details_total_unique_applicants,
    "term_2_details_total_applications": term_2_details_total_applications,
    "term_2_total_applications_percent_change_number_unique_absolute": term_2_total_applications_percent_change_number_unique_absolute,
    "higher_lower_2": higher_lower_2,
    "term_2_details_accepted_applications": term_2_details_accepted_applications,
    "term_2_acceptance_percentage_number": term_2_acceptance_percentage_number,
    "term_2_details_international_applications": term_2_details_international_applications,
    "term_2_international_percentage_number": term_2_international_percentage_number,
    "term_3_details_total_unique_applicants": term_3_details_total_unique_applicants,
    "term_3_details_total_applications": term_3_details_total_applications,
    "term_3_total_applications_percent_change_number_unique_absolute": term_3_total_applications_percent_change_number_unique_absolute,
    "higher_lower_3": higher_lower_3,
    "term_3_details_accepted_applications": term_3_details_accepted_applications,
    "term_3_acceptance_percentage_number": term_3_acceptance_percentage_number,
    "term_3_details_international_applications": term_3_details_international_applications,
    "term_3_international_percentage_number": term_3_international_percentage_number,

    #Briefing Note Enrolment Map
    "Enrolment_current_academic_year": variable_Current_enrolment_academic_year,
    "Enrolment_header_1": converted_Extraction_date,
    "Enrolment_header_2": variable_Current_enrolment_academic_year,
    "Enrolment_header_3": previous_year_Extraction_Date,
    "Enrolment_header_4": previous_year_enrolment_academic_year,
    "Enrolment_box_1": enrolment_Total_UHC,
    "Enrolment_box_2": enrolment_International_UHC,
    "Enrolment_box_3": enrolment_Domestic_UHC,
    "Enrolment_box_4": enrolment_Indigenous_UHC,
    "Enrolment_box_5": enrolment_Apprenticeship_UHC,
    "Enrolment_box_6": enrolment_Certificate_UHC,
    "Enrolment_box_7": enrolment_Diploma_UHC,
    "Enrolment_box_8": enrolment_Non_Credential_UHC,
    "Enrolment_box_9": enrolment_Fall_UHC,
    "Enrolment_box_10": enrolment_Winter_UHC,
    "Enrolment_box_11": f"{round(enrolment_Total_FLE, 3):.3f}",
    "Enrolment_box_12": f"{round(enrolment_International_FLE, 3):.3f}",
    "Enrolment_box_13": f"{round(enrolment_Domestic_FLE, 3):.3f}",
    "Enrolment_box_14": f"{round(enrolment_Indigenous_FLE, 3):.3f}",
    "Enrolment_box_15": f"{round(enrolment_Apprenticeship_FLE, 3):.3f}",
    "Enrolment_box_16": f"{round(enrolment_Certificate_FLE, 3):.3f}",
    "Enrolment_box_17": f"{round(enrolment_Diploma_FLE, 3):.3f}",
    "Enrolment_box_18": f"{round(enrolment_Non_Credential_FLE, 3):.3f}",
    "Enrolment_box_19": f"{round(enrolment_Fall_FLE, 3):.3f}",
    "Enrolment_box_20": f"{round(enrolment_Winter_FLE, 3):.3f}",
    "Enrolment_box_21": previous_enrolment_Total_UHC,
    "Enrolment_box_22": previous_enrolment_International_UHC,
    "Enrolment_box_23": previous_enrolment_Domestic_UHC,
    "Enrolment_box_24": previous_enrolment_Indigenous_UHC,
    "Enrolment_box_25": previous_enrolment_Apprenticeship_UHC,
    "Enrolment_box_26": previous_enrolment_Certificate_UHC,
    "Enrolment_box_27": previous_enrolment_Diploma_UHC,
    "Enrolment_box_28": previous_enrolment_Non_Credential_UHC,
    "Enrolment_box_29": previous_enrolment_Fall_UHC,
    "Enrolment_box_30": previous_enrolment_Winter_UHC,
    "Enrolment_box_31": f"{round(previous_enrolment_Total_FLE, 3):.3f}",
    "Enrolment_box_32": f"{round(previous_enrolment_International_FLE, 3):.3f}",
    "Enrolment_box_33": f"{round(previous_enrolment_Domestic_FLE, 3):.3f}",
    "Enrolment_box_34": f"{round(previous_enrolment_Indigenous_FLE, 3):.3f}",
    "Enrolment_box_35": f"{round(previous_enrolment_Apprenticeship_FLE, 3):.3f}",
    "Enrolment_box_36": f"{round(previous_enrolment_Certificate_FLE, 3):.3f}",
    "Enrolment_box_37": f"{round(previous_enrolment_Diploma_FLE, 3):.3f}",
    "Enrolment_box_38": f"{round(previous_enrolment_Non_Credential_FLE, 3):.3f}",
    "Enrolment_box_39": f"{round(previous_enrolment_Fall_FLE, 3):.3f}",
    "Enrolment_box_40": f"{round(previous_enrolment_Winter_FLE, 3):.3f}",
    "Enrolment_box_41": enrolment_Total_percent_change_UHC,
    "Enrolment_box_42": enrolment_International_percent_change_UHC,
    "Enrolment_box_43": enrolment_Domestic_percent_change_UHC,
    "Enrolment_box_44": enrolment_Indigenous_percent_change_UHC,
    "Enrolment_box_45": enrolment_Apprenticeship_percent_change_UHC,
    "Enrolment_box_46": enrolment_Certificate_percent_change_UHC,
    "Enrolment_box_47": enrolment_Diploma_percent_change_UHC,
    "Enrolment_box_48": enrolment_Non_Credential_percent_change_UHC,
    "Enrolment_box_49": enrolment_Fall_percent_change_UHC,
    "Enrolment_box_50": enrolment_Winter_percent_change_UHC,
    "Enrolment_box_51": enrolment_Total_percent_change_FLE,
    "Enrolment_box_52": enrolment_International_percent_change_FLE,
    "Enrolment_box_53": enrolment_Domestic_percent_change_FLE,
    "Enrolment_box_54": enrolment_Indigenous_percent_change_FLE,
    "Enrolment_box_55": enrolment_Apprenticeship_percent_change_FLE,
    "Enrolment_box_56": enrolment_Certificate_percent_change_FLE,
    "Enrolment_box_57": enrolment_Diploma_percent_change_FLE,
    "Enrolment_box_58": enrolment_Non_Credential_percent_change_FLE,
    "Enrolment_box_59": enrolment_Fall_percent_change_FLE,
    "Enrolment_box_60": enrolment_Winter_percent_change_FLE,
    "Enrolment_current_academic_year": variable_Current_enrolment_academic_year,
    
    "Enrolment_total_FLE_projection": f"{round(enrolment_projection_map["FLE Projection All Students"], 3):.3f}",
    "Enrolment_total_FLE": f"{round(enrolment_Total_FLE, 3):.3f}",
    "a_an_1": a_an_1,
    "Enrolment_FLE_percent_change_number_absolute": Enrolment_FLE_percentage_number_absolute,
    "surpass_NULL_1": surpass_NULL_1,
    "Enrolment_term_year_1": Enrolment_term_year_1,
    "Enrolment_term_year_1_part_time_HC_projection": Enrolment_term_year_1_part_time_HC_projection,
    "Enrolment_term_year_1_part_time_HC": Enrolment_term_year_1_part_time_HC,
    "a_an_2": a_an_2,
    "Enrolment_term_year_1_part_time_HC_percent_change_number_absolute": Enrolment_term_year_1_part_time_HC_percent_change_number_absolute,
    "surpass_NULL_2": surpass_NULL_2,
    "Enrolment_term_year_1_full_time_HC_projection": Enrolment_term_year_1_full_time_HC_projection,
    "Enrolment_term_year_1_full_time_HC": Enrolment_term_year_1_full_time_HC,
    "a_an_3": a_an_3,
    "Enrolment_term_year_1_full_time_HC_percent_change_number_absolute": Enrolment_term_year_1_full_time_HC_percent_change_number_absolute,
    "surpass_NULL_3": surpass_NULL_3,
    "Enrolment_term_year_2": Enrolment_term_year_2,
    "Enrolment_term_year_2_part_time_HC_projection": Enrolment_term_year_2_part_time_HC_projection,
    "Enrolment_term_year_2_part_time_HC": Enrolment_term_year_2_part_time_HC,
    "a_an_4": a_an_4,
    "Enrolment_term_year_2_part_time_HC_percent_change_number_absolute": Enrolment_term_year_2_part_time_HC_percent_change_number_absolute,
    "surpass_NULL_4": surpass_NULL_4,
    "Enrolment_term_year_2_full_time_HC_projection": Enrolment_term_year_2_full_time_HC_projection,
    "Enrolment_term_year_2_full_time_HC": Enrolment_term_year_2_full_time_HC,
    "a_an_5": a_an_5,
    "Enrolment_term_year_2_full_time_HC_percent_change_number_absolute": Enrolment_term_year_2_full_time_HC_percent_change_number_absolute,
    "surpass_NULL_5": surpass_NULL_5,
    "Enrolment_LINC_last_intake_term_year": Enrolment_LINC_last_intake_term_year,

    "Enrolment_box_61": enrolment_projection_map["Summer Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_62": enrolment_projection_map["Summer Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_63": enrolment_projection_map["Summer Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_64": enrolment_projection_map["Summer Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_65": enrolment_full_International_Summer_exclude_CML_LINC_APPR,
    "Enrolment_box_66": enrolment_part_International_Summer_exclude_CML_LINC_APPR,
    "Enrolment_box_67": enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR,
    "Enrolment_box_68": enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR,
    "Enrolment_box_69": "N/A" if enrolment_full_International_Summer_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_International_Summer_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_70": "N/A" if enrolment_part_International_Summer_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_International_Summer_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_71": "N/A" if enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_Domestic_Summer_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_72": "N/A" if enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_Domestic_Summer_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_73": enrolment_projection_map["Fall Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_74": enrolment_projection_map["Fall Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_75": enrolment_projection_map["Fall Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_76": enrolment_projection_map["Fall Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_77": enrolment_full_International_Fall_exclude_CML_LINC_APPR,
    "Enrolment_box_78": enrolment_part_International_Fall_exclude_CML_LINC_APPR,
    "Enrolment_box_79": enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR,
    "Enrolment_box_80": enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR,
    "Enrolment_box_81": "N/A" if enrolment_full_International_Fall_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_International_Fall_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_82": "N/A" if enrolment_part_International_Fall_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_International_Fall_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_83": "N/A" if enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_Domestic_Fall_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_84": "N/A" if enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_Domestic_Fall_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_85": enrolment_projection_map["Winter Full-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_86": enrolment_projection_map["Winter Part-time Headcount Projection International Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_87": enrolment_projection_map["Winter Full-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_88": enrolment_projection_map["Winter Part-time Headcount Projection Domestic Students (Excluding CML/LINC/APPR)"],
    "Enrolment_box_89": enrolment_full_International_Winter_exclude_CML_LINC_APPR,
    "Enrolment_box_90": enrolment_part_International_Winter_exclude_CML_LINC_APPR,
    "Enrolment_box_91": enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR,
    "Enrolment_box_92": enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR,
    "Enrolment_box_93": "N/A" if enrolment_full_International_Winter_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_International_Winter_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_94": "N/A" if enrolment_part_International_Winter_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_International_Winter_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_95": "N/A" if enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_full_Domestic_Winter_exclude_CML_LINC_APPR_percentage,
    "Enrolment_box_96": "N/A" if enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR_percentage == "N/A%" else enrolment_part_Domestic_Winter_exclude_CML_LINC_APPR_percentage,

    "Enrolment_term_year_1_Apprenticeship_UHC_projection": enrolment_projection_map_Fall_Apprenticeship["Total"],
    "Enrolment_term_year_1_Apprenticeship_UHC": Enrolment_term_year_1_Apprenticeship_HC,
    "a_an_6": a_an_6,
    "Enrolment_term_year_1_Apprenticeship_UHC_percent_change_number_absolute": Enrolment_term_year_1_Apprenticeship_HC_percent_change_number_absolute,
    "surpass_NULL_6": surpass_NULL_6,
    "Enrolment_term_year_1_Apprenticeship_bullet_point_programs": enrolment_Aprrenticeship_Fall_achieved_surpassed_bullet_points,
    "Enrolment_term_year_2_Apprenticeship_UHC_projection": enrolment_projection_map_Winter_Apprenticeship["Total"],
    "Enrolment_term_year_2_Apprenticeship_UHC": Enrolment_term_year_2_Apprenticeship_HC,
    "a_an_7": a_an_7,
    "Enrolment_term_year_2_Apprenticeship_UHC_percent_change_number_absolute": Enrolment_term_year_2_Apprenticeship_HC_percent_change_number_absolute,
    "surpass_NULL_7": surpass_NULL_7,
    "Enrolment_term_year_2_Apprenticeship_bullet_point_programs": enrolment_Aprrenticeship_Winter_achieved_surpassed_bullet_points,

    "Enrolment_International_UHC": enrolment_International_UHC,
    "Enrolment_International_FLE": f"{round(enrolment_International_FLE, 3):.3f}",
    "Enrolment_International_rank_1_program": rank_1_International_program_name,
    "rank_sentence": rank_sentence,
    "Enrolment_International_FLE_percentage_number": enrolment_International_FLE_percentage,

    "Enrolment_Indigenous_UHC": enrolment_Indigenous_UHC,
    "Enrolment_Indigenous_FLE": f"{round(enrolment_Indigenous_FLE, 3):.3f}",
    "Enrolment_Indigenous_rank_1_program": rank_1_Indigenous_program_name,
    "Enrolment_Indigenous_FLE_percentage_number": enrolment_Indigenous_FLE_percentage
}

# Utility: check if a value is negative
def is_negative(value):
    if isinstance(value, (int, float)):
        return value < 0
    if isinstance(value, str):
        try:
            return float(value.strip('%')) < 0
        except:
            return False
    return False

# Prepare context with red formatting for negative values
context = {}
for key, value in raw_data.items():
    if is_negative(value):
        # Apply red color to negative values
        context[key] = f"<w:r><w:rPr><w:color w:val='FF0000' /></w:rPr><w:t>{value}</w:t></w:r>"
    else:
        context[key] = value

# Render the template with context
doc.render(context)

# Define output folder, create one if it doesn't exist
output_folder = "Output Files"
os.makedirs(output_folder, exist_ok=True)

# Save output to a new document
safe_date = re.sub(r'[\\/*?:"<>|]', "_", variable_Extraction_date)
briefing_note_file_name = f"Briefing Note - KC Applications and Enrolments KPI Update as of {safe_date}.docx"
output_path = os.path.join(output_folder, briefing_note_file_name)
doc.save(output_path)